<a href="https://colab.research.google.com/github/benzionchen/transformer_NLP_research/blob/main/hugging_face_chapter_1_3_codealong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
!nvidia-smi

Tue Mar 25 05:49:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P0             31W /   70W |     420MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install transformers torch

In [2]:
import transformers

In [3]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis')
classifier("I've been waiting for Hugging Face course")

# pipeline() is function provided by HF, using their api to load pretrained model and run inference (text gen, sentiment analysis, etc.)
# it's doing model loading -> tokenization -> prediction in 1 line of code (defaults to distilbert-base-uncased-finetuned-sst-2-english model)
# what's happening to inference when you call pipeline() in inference time? the function is called, the pipeline is initialized (loads tokenizer and model weights) using pytorch or tensorflow setting up the inference backend, then GPU is enabled in colab
# which will run locally on my

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.995071530342102}]

In [4]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

# sentiment analysis is determining tone behind speech, positive and negative connotation + confidence level (confidence will never be 1.0 or 100%)
# the output is coming from input tensors that are fed into the pretrained model, so the model produces logits -> softmax -> classification probabilities
# model weights are downloaded from HF the first time + inference is done on colab's machine (VM probably at google's datacenter and not using local hardware like my 3080ti)

# 3 main steps involved when passing text into pipeline: 1. text is preprocessed into format model can understand 2. inputs are passed to model 3. predictions are post-processed so you can make sense of them
# format = vectors (tensors), matmul happens on these tensors, tokens are mapped to numbers via tokenizer, token ID used to index an embedding matrix to get vectors, turning words into vectors
# vectors are passed through layers of the transformer (inside attention layer + feedforward networks use matmul operations)
# attention score = softmax(QK transpose / root(d_k))

[{'label': 'POSITIVE', 'score': 0.9598046541213989},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [5]:
classifier = pipeline('zero-shot-classification')

classifier(
    "this is a course on transformers",
    candidate_labels = ["education", "politics", "business"],
)
# my scores are different than the example in HF is that due to change in model weights? model drift, weights updated over time, new variants of the model, improving tokenization + processing logic can change scores, etc.

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


{'sequence': 'this is a course on transformers',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8483402729034424, 0.12006837129592896, 0.03159131482243538]}

In [7]:
print(classifier.model.name_or_path)

# zero shot = user doesnt have to fine-tune of the model

facebook/bart-large-mnli


In [8]:
generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

# The main idea here is that you provide a prompt and the model will auto-complete it by generating the remaining text. This is similar to the predictive text feature that is found on many phones. Text generation involves randomness

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to develop a basic understanding of the principles, techniques, and techniques of the Chinese language, the way that using Chinese language is used, and how to become fluent in all types of Chinese. We will also'}]

In [9]:
# try with r1? might run out of space

generator = pipeline(
    "text-generation",
    model="deepseek-ai/deepseek-llm-7b-base",
    device_map="auto",        # use GPU if available
    trust_remote_code=True    # custom model logic
)

output = generator("In this course, we will teach you how to", max_new_tokens=50)
print(output[0]["generated_text"])

# runs way slower than the one above

config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


ValueError: Could not load model deepseek-ai/deepseek-llm-7b-base with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py", line 290, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py", line 262, in _wrapper
    return func(*args

In [10]:
# big language models (like DeepSeek or LLaMA) are too massive to store in a single file (sometimes >20GB). So they're sharded — split into smaller chunks (e.g., pytorch_model-00001-of-00003.bin, pytorch_model-00002-of-00003.bin
# .bin files contain PyTorch tensors (learned weights of the model), .bin files are frozen brains of the model — all the neurons' values are saved here

# how are the model weights stored in bin? weights are FP32 usually and stored as byte sequence in bin using pytorch serialization tensor is stored as bin blob of floats
# looks like {
#  'encoder.layer.0.attention.self.query.weight': tensor([...]),
#  'encoder.layer.0.attention.self.key.weight': tensor([...]),
#  'encoder.layer.0.attention.self.value.weight': tensor([...]),
#  ...
#}

# each tensor is big matrix of floats stored as bin data, need pytorch to deserialize - this is different than an instruction set

In [11]:
import torch

# load the binary weight file
state_dict = torch.load("(insert pytorch_model.bin)") # this would probably be proprietary?

# list all parameter names
print(state_dict.keys())

# see one weight tensor
print(state_dict['encoder.layer.0.attention.self.query.weight'])

#probably will output soemthing like:
# tensor([[ 0.023, -0.042, ..., 0.019],
#         [-0.001,  0.112, ..., -0.055], ...])

FileNotFoundError: [Errno 2] No such file or directory: 'pytorch_model.bin'

In [12]:
generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to build the best team in the world.\n\n\n\nThere were times that I wanted to learn'},
 {'generated_text': 'In this course, we will teach you how to apply different concepts to different people. This course is about teaching a lot about how to design a business'}]

In [13]:
unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

# <mask> is like "(insert here)" and the model will predict it

# different than prefill, prefill = decoder-only LLMs like GPT and Deepseek where first pass of model encodes input context prompt into internal hidden states, prepping model memory before generation
# modern inference systems like vLLM use prefill (encode the input text) + decode (generate new tokens)

# top_k argument controls how many possibilities you want to be displayed

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


[{'score': 0.1919853836297989,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical models.'},
 {'score': 0.04209202527999878,
  'token': 38163,
  'token_str': ' computational',
  'sequence': 'This course will teach you all about computational models.'}]

In [14]:
ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

# named entity recognition - NER = task where model ahs to find which parts of the input text corresponds to entities such as person(s) + location(s) + organization(s)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


[{'entity_group': 'PER',
  'score': np.float32(0.9981694),
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': np.float32(0.9796019),
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': np.float32(0.9932106),
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]